## 智能体评估 - 模拟人类
***

创建一个简单的智能体

In [13]:

from langchain_deepseek import ChatDeepSeek
from dotenv import load_dotenv
import os
# 加载 .env 文件
load_dotenv()

model =  ChatDeepSeek(
    model="Pro/deepseek-ai/DeepSeek-V3",
    temperature=0,
    api_key=os.environ.get("DEEPSEEK_API_KEY"),
    base_url=os.environ.get("DEEPSEEK_API_BASE"),
)

from typing import Literal

from langchain_core.tools import tool


@tool
def get_tuikuan_zhengce():
    """查询退款政策"""
    return "机票退款时间为购票后24小时内，且航班起飞前48小时，可全额退款。"


tools = [get_tuikuan_zhengce]


from langgraph.prebuilt import create_react_agent
prompt = "你是一名航空公司的客服人员。"
graph = create_react_agent(model, tools=tools,prompt=prompt)

In [14]:
from langchain_core.messages import HumanMessage
def chat_bot(query: str):
    config = {"configurable": {"thread_id": "2"}}
    res = graph.invoke({"messages":[HumanMessage(query)]}, config)
    return res

In [15]:
chat_bot("你好啊！")

{'messages': [HumanMessage(content='你好啊！', additional_kwargs={}, response_metadata={}, id='b556ba0b-06d8-4aff-864c-7e62cb77ebc4'),
  AIMessage(content='你好！很高兴为您服务。请问有什么可以帮您的吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 81, 'total_tokens': 94, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Pro/deepseek-ai/DeepSeek-V3', 'system_fingerprint': '', 'id': '0195fa52e5f81cc88c890d41677a9bae', 'finish_reason': 'stop', 'logprobs': None}, id='run-12b16cb0-aaf6-45d0-9326-b759c1a9acff-0', usage_metadata={'input_tokens': 81, 'output_tokens': 13, 'total_tokens': 94, 'input_token_details': {}, 'output_token_details': {}})]}

定义模拟的真实用户

In [16]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# 系统提示模板：定义客户与航空公司客服人员的交互场景
system_prompt_template = """你是航空公司的一名客户。\
你正在与一位客服人员进行交互。\

{instructions}

当你结束对话时，只需回复一个单词 'FINISHED'"""

# 创建聊天提示模板
prompt = ChatPromptTemplate.from_messages(
    [
        # 系统消息提示：使用系统提示模板
        ("system", system_prompt_template),
        # 消息占位符：用于动态插入用户与 AI 的消息
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# 指令：定义客户的具体身份和目标
instructions = """你的名字是 Tomie。你正在试图为一次去阿拉斯加的旅行申请退款。\
你希望他们退还所有的钱。\
这次旅行发生在 5 年前。"""

# 将指令和客户姓名部分应用到提示模板中
prompt = prompt.partial(name="Tomie", instructions=instructions)

# 创建语言模型实例
model = ChatDeepSeek(
    model="Pro/deepseek-ai/DeepSeek-V3",
    temperature=0,
    api_key=os.environ.get("DEEPSEEK_API_KEY"),
    base_url=os.environ.get("DEEPSEEK_API_BASE"),
)

# 模拟用户：将提示模板与模型连接，用于生成模拟的用户交互
simulated_user = prompt | model


测试真实用户反馈

In [17]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(content="你好，有什么可以帮助你的吗?")]
simulated_user.invoke({"messages": messages})

AIMessage(content='你好，我是Tomie。我想申请5年前去阿拉斯加的机票退款。我需要全额退款。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 71, 'total_tokens': 92, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Pro/deepseek-ai/DeepSeek-V3', 'system_fingerprint': '', 'id': '0195fa5446b7967973104e47c93d8d94', 'finish_reason': 'stop', 'logprobs': None}, id='run-364f69ed-04c2-4769-a238-375e687117d2-0', usage_metadata={'input_tokens': 71, 'output_tokens': 21, 'total_tokens': 92, 'input_token_details': {}, 'output_token_details': {}})

使用langgraph来进行模拟，首先定义节点

In [18]:
from langchain_core.messages import AIMessage


def chat_bot_node(state):
    messages = state["messages"]
    # 调用客服机器人
    if len(messages)>0:
        chat_bot_response = chat_bot(messages[-1].content)
        response_content = chat_bot_response["messages"][-1].content
        # 使用 AI 消息进行响应
        return {"messages": [AIMessage(content=response_content)]}


定义模拟用户节点

In [9]:
def _swap_roles(messages):
    new_messages = []
    for m in messages:
        # 如果消息是 AIMessage，则将其转换为 HumanMessage
        if isinstance(m, AIMessage):
            new_messages.append(HumanMessage(content=m.content))
        # 否则，将其转换为 AIMessage
        else:
            new_messages.append(AIMessage(content=m.content))
    return new_messages


def simulated_user_node(state):
    messages = state["messages"]
    # 交换消息的角色（AI 和用户角色互换）
    new_messages = _swap_roles(messages)
    # 调用模拟用户
    response = simulated_user.invoke({"messages": new_messages})
    # 这个响应是一个 AI 消息——我们需要将其转换为用户消息
    return {"messages": [HumanMessage(content=response.content)]}


定义条件边，结束对话的条件

In [ ]:
def should_continue(state):
    messages = state["messages"]
    # 如果消息数量大于 10，则结束
    if len(messages) > 10:
        return "end"
    elif messages[-1].content == "FINISHED":
        return "end"
    else:
        return "continue"

定义graph

In [19]:
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)
graph_builder.add_node("user", simulated_user_node)
graph_builder.add_node("chat_bot", chat_bot_node)
# 聊天机器人的每一个响应都会自动发送给模拟用户
graph_builder.add_edge("chat_bot", "user")
graph_builder.add_conditional_edges(
    "user",
    should_continue,
    # 如果满足结束条件，我们将停止模拟，
    # 否则，虚拟用户的消息将被发送给聊天机器人
    {
        "end": END,
        "continue": "chat_bot",
    },
)
# 输入将首先发送给聊天机器人
graph_builder.add_edge(START, "chat_bot")
simulation = graph_builder.compile()


运行模拟

In [20]:
for chunk in simulation.stream({"messages": "你好！"}):
    if END not in chunk:
        print(chunk)
        print("----")

{'chat_bot': {'messages': [AIMessage(content='您好！我是航空公司的客服人员，很高兴为您服务。请问有什么可以帮您的吗？', additional_kwargs={}, response_metadata={}, id='dff1357f-e844-4e62-ba9c-90470948056d')]}}
----
{'user': {'messages': [HumanMessage(content='你好！我是Tomie。我想申请退款，5年前我乘坐了你们航空公司去阿拉斯加的航班。现在我需要全额退款。', additional_kwargs={}, response_metadata={}, id='fc2844b0-126b-4c94-aa0b-9231786fe08d')]}}
----
{'chat_bot': {'messages': [AIMessage(content='稍等，我帮您查询一下相关的退款政策。由于您的航班已经是5年前的事情，我需要确认一下是否还在退款的有效期内。', additional_kwargs={}, response_metadata={}, id='fcbdfa37-638f-400d-ae2c-339c20ff6ccb')]}}
----
{'user': {'messages': [HumanMessage(content='(不耐烦地)有效期？都5年了当然不在有效期内！但这不是重点，我就是要退款。你们航空公司欠我的！', additional_kwargs={}, response_metadata={}, id='42e08714-b3cf-4dea-91e7-bd785d1b7f8c')]}}
----
{'chat_bot': {'messages': [AIMessage(content='我理解您对退款问题的强烈情绪，但根据航空公司的政策，退款通常需要在机票的有效期内申请。如果您的机票已经过期5年，可能无法直接通过常规流程退款。\n\n不过，如果您认为航空公司有其他特殊情况或责任导致您需要退款，您可以提供更多细节（例如航班取消、延误或其他问题），我会尽力帮您查询是否有其他解决方案。\n\n如果您愿意，我可以为您查询相关的退款政策或建议下一步的操作。请告诉我您的具体情况。'